### Import libraries

In [84]:
import hdlparse.verilog_parser as vlog
import re
import random

### open and read verilog file

In [85]:
vlog_ex = vlog.VerilogExtractor()
fname = 'Module1.v'
with open(fname, 'rt') as fh:
  code = fh.read()
vlog_mods = vlog_ex.extract_objects_from_source(code)

vlog_mods = vlog_ex.extract_objects(fname)

### print module name and ports

In [87]:
def vector_size(line):
	if re.search(r'\[', line):
		lval = re.findall(r'\[(.*):', line)[0]
		rval = re.findall(r'\[.*:(.*)\]', line)[0]
		return (abs(int(rval) - int(lval)) + 1)
	return 1

### add each inputs and outputs in list as dictionary

In [89]:
for m in vlog_mods:
  print('Module "{}":'.format(m.name))
  inputs_list = []
  ports_list = []
  print('\n  Ports:')
  for p in m.ports:
        if (p.data_type).find('[') != -1 :
            inputs_list.append({"name":p.name ,"mode":p.mode ,"data_type":(p.data_type).split()[0],"length":vector_size((p.data_type).split()[1]) })
        else :
            inputs_list.append({"name":p.name ,"mode":p.mode ,"data_type":p.data_type, "length": 1})
        print('\t{:20}{:8}{}'.format(p.name, p.mode, p.data_type))



  Ports:
	HasAccess           input   wire
	Input_Pin           input   reg [3:0]
	message             output  reg [3:0]
	Welcome             output  reg


### generate vector function generate the vector if size > 1

In [101]:
def generate_vector(port_length):
    if port_length == 1:
        return ' '
    else:
        return (' [' + str(port_length - 1) + ':0] ')

In [102]:
def generate_range(port_length):
    if port_length == 1:
        return '1'
    else:
        return str(pow(2,port_length)-1)

### add testbench content in string

In [103]:
def testbench_header_and_ports():
    content = ""
    content += ('module ' + m.name + '_tb();\n')
    for port in inputs_list:
        if port["mode"].find('input', 0, 5) != -1:
            content += ('    ' + port["mode"].replace('input', 'reg') +  generate_vector(port["length"]) + port["name"] + '_tb;\n')
        elif port["mode"].find('output', 0, 6) != -1:
            content += ('    ' + port["mode"].replace('output', 'wire') + generate_vector(port["length"]) + port["name"] +  '_tb;\n')
    return content
    

In [104]:
def testbench_initial_block(time_delay):
    global val
    content = ""
    content += ('  integer i;\n  begin\n//initial block\ninitial\n  begin\n    //initial values\n')
    maxPortSize = 12
    for port in inputs_list :
        maxPortSize = max(port["length"],maxPortSize)
    iterationCounter = int(pow(2,maxPortSize)/2)
    iterationCounter = min(maxPortSize, 50)
    for i in range (iterationCounter):
        if(i != 0):
            content += ('  #'+ str(time_delay)+'\n')
        for line in inputs_list:
            if line["mode"] == 'input':
                val = line["length"]
                content += ('    ' + line["name"] + '_tb = '+ str(val) + '\'b'+ bin(random.randint(0, pow(2,val)-1))[2:].zfill(val)+ ';' + '\n')

    content += ('\n//Random generated constraints\n')
    content += ('for(i = 0; i < 100; i = i+1)\n  begin\n  #'+str(time_delay)+'\n')
    for line in inputs_list:
            if line["mode"] == 'input':
                val = line["length"]
                content += ('    ' + line["name"] + '_tb = '+ '$urandom_range(0,'+generate_range(val) + ');' + '\n')
    content+= ('  end\n')
    content += ('  $finish;\nend\n')
    return content

In [106]:
def testbench_design_instance():
    content = ""
    content += ('// instaniate design instance\n  ' + m.name + ' DUT (\n')
    for i,line in enumerate(inputs_list, 0):
        content += ('    .' + (line["name"]).replace('_tb', '') + '(' + line["name"]+ '_tb'+')')
        if i != len(inputs_list)-1:
            content += (',\n')
        else:
            content += ('\n);\n')

    return content

In [107]:
def testbench_monitor_block() :
    content = ""
    word_size = []
    content += ('initial begin\n  $display ("        			","Time ')
    for port in inputs_list :
        content += (port["name"]+ ' ')
        word_size.append(len(port["name"]))
    content += ('");\n')
    content += ('  $monitor("    %d')
    for i in range(1, len(inputs_list)+1) :
        w = max(word_size[i-1], port["length"])
        s = '{space:{width}}{id}'.format(space = ' ', width = w+1, id = '%d')
        content += (s)
    content += ('" , $time')
    for port in inputs_list :
        content += (','+ port["name"]+'_tb')
    content += (');\n')
    content += ('end\n')
    content += ('end\nendmodule')
    return content

### testbench file generation

In [109]:
def generate_output_file(file_content):
    output_file = fname.replace('.v', '_tb.v') 
    with open(output_file, 'w') as f:
            f.write(file_content)

### Testbench generator

In [110]:
time_delay = 10 #input or not
test_bench_content = ""
test_bench_content += testbench_header_and_ports() + testbench_initial_block(time_delay) + testbench_design_instance() + testbench_monitor_block()
generate_output_file(test_bench_content)